In [29]:
from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [30]:
data_dir = "../DATA/"
tests_dir = "../TESTS/"

In [31]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [32]:
def read_as_dataframe(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = f[gamma].keys()
            
            for state in init_states:
                
                F_lme_exact = f[gamma][state]["F_lme_exact"][()]
                F_sidlme_lme_rnd = f[gamma][state]["F_sidlme_lme_rnd"][()]
                F_sidlme_lme_tssos = f[gamma][state]["F_sidlme_lme_tssos"][()]

                F_sidexact_exact_rnd = f[gamma][state]["F_sidexact_exact_rnd"][()]
                F_sidexact_exact_tssos = f[gamma][state]["F_sidexact_exact_tssos"][()]
                
                
                ser_len = len(F_lme_exact)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                time_column = t
                gamma_time_column = np.array(t)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'F_lme_exact': F_lme_exact,
                         'F_sidlme_lme_rnd': F_sidlme_lme_rnd,
                         'F_sidlme_lme_tssos': F_sidlme_lme_tssos,
                         'F_sidexact_exact_rnd': F_sidexact_exact_rnd,
                         'F_sidexact_exact_tssos': F_sidexact_exact_tssos,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [33]:
test_file = "POP_LME_tssos_vs_random_trn4_tst20_fix-Phi-up_2024-Jan-09_at_14-34.h5"

In [34]:
df = read_as_dataframe(test_file, fid_dists, data_dir, tests_dir)

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [35]:
pkl_name = "dataframe_"+test_file[:-3]+".pkl"

df.to_pickle(tests_dir+pkl_name)

In [36]:
pkl_name

'dataframe_POP_LME_tssos_vs_random_trn4_tst20_fix-Phi-up_2024-Jan-09_at_14-34.pkl'